In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [4]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [5]:
from allennlp.common.checks import ConfigurationError

In [6]:
USE_GPU = torch.cuda.is_available()

In [19]:
DATA_ROOT = Path("./data") / "jigsaw"

Set random seed manually to replicate results

In [8]:
torch.manual_seed(config.seed)

# Load Data

In [9]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [10]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [11]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [16]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import SingleIdTokenIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = SingleIdTokenIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [17]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [20]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None



0it [00:00, ?it/s]

1it [00:00,  1.67it/s]

16it [00:00,  2.38it/s]
38it [00:00,  3.37it/s]
58it [00:00,  4.78it/s]
76it [00:01,  6.75it/s]
99it [00:01,  9.52it/s]
125it [00:01, 13.39it/s]
144it [00:01, 18.40it/s]
163it [00:01, 25.14it/s]
193it [00:01, 34.63it/s]
215it [00:01, 46.04it/s]
241it [00:01, 60.78it/s]
265it [00:01, 78.26it/s]
267it [00:01, 136.85it/s]
0it [00:00, ?it/s]
18it [00:00, 169.85it/s]
42it [00:00, 184.89it/s]
61it [00:00, 185.42it/s]
84it [00:00, 196.28it/s]
114it [00:00, 215.52it/s]
136it [00:00, 216.12it/s]
157it [00:00, 205.88it/s]
183it [00:00, 217.62it/s]
209it [00:00, 228.64it/s]
232it [00:01, 227.90it/s]
251it [00:01, 216.13it/s]

In [21]:
len(train_ds)

267

In [22]:
train_ds[:10]

In [23]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x7f8c3e00ab38>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

In [24]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)


100%|██████████| 267/267 [00:00<00:00, 12191.42it/s]

### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [25]:
from allennlp.data.iterators import BucketIterator

In [26]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [27]:
iterator.index_with(vocab)

### Read sample

In [28]:
batch = next(iter(iterator(train_ds)))

In [29]:
batch

{'tokens': {'tokens': tensor([[   5,    5,    5,  ...,   21,  558,    8],
          [   5,  621,  764,  ...,   84,   15,    4],
          [   5,  224,   19,  ...,  159,   13,  122],
          ...,
          [   5, 1121,  428,  ...,   14,    2,   86],
          [   5,   74,   17,  ...,    3, 3337,   72],
          [ 213, 1658,  828,  ..., 1671, 1672,    6]])},
 'id': ['002746baedcdff10',
  '006de7a80921e04b',
  '0048de0c9422f64f',
  '005cec874506e9d9',
  '0006f16e4e9f292e',
  '006d11791d76b9f3',
  '008a1e9c45de8138',
  '00148d055a169b93',
  '00905910dcbcc8aa',
  '00a66540b2a66bb4',
  '0021fe88bc4da3e6'],
 'label': tensor([[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 

In [30]:
batch["tokens"]["tokens"]

tensor([[   5,    5,    5,  ...,   21,  558,    8],
        [   5,  621,  764,  ...,   84,   15,    4],
        [   5,  224,   19,  ...,  159,   13,  122],
        ...,
        [   5, 1121,  428,  ...,   14,    2,   86],
        [   5,   74,   17,  ...,    3, 3337,   72],
        [ 213, 1658,  828,  ..., 1671, 1672,    6]])

In [31]:
batch["tokens"]["tokens"].shape

torch.Size([11, 100])

# Prepare Model

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

In [33]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [34]:
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

token_embedding = Embedding(num_embeddings=config.max_vocab_size + 2,
                            embedding_dim=300, padding_index=0)
# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

In [35]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(),
                                                        config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [36]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [37]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [38]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [39]:
tokens = batch["tokens"]
labels = batch

In [40]:
tokens

{'tokens': tensor([[   5,    5,    5,  ...,   21,  558,    8],
         [   5,  621,  764,  ...,   84,   15,    4],
         [   5,  224,   19,  ...,  159,   13,  122],
         ...,
         [   5, 1121,  428,  ...,   14,    2,   86],
         [   5,   74,   17,  ...,    3, 3337,   72],
         [ 213, 1658,  828,  ..., 1671, 1672,    6]])}

In [41]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [42]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[-0.0371,  0.0806, -0.0309,  0.0270,  0.0156, -0.0257],
        [-0.0385,  0.0808, -0.0315,  0.0275,  0.0149, -0.0257],
        [-0.0379,  0.0798, -0.0332,  0.0267,  0.0166, -0.0253],
        [-0.0387,  0.0787, -0.0305,  0.0272,  0.0173, -0.0241],
        [-0.0396,  0.0810, -0.0299,  0.0276,  0.0167, -0.0256],
        [-0.0390,  0.0815, -0.0305,  0.0276,  0.0160, -0.0249],
        [-0.0371,  0.0797, -0.0311,  0.0261,  0.0166, -0.0240],
        [-0.0378,  0.0807, -0.0309,  0.0268,  0.0151, -0.0257],
        [-0.0388,  0.0800, -0.0314,  0.0275,  0.0159, -0.0246],
        [-0.0379,  0.0802, -0.0313,  0.0283,  0.0164, -0.0240],
        [-0.0382,  0.0797, -0.0317,  0.0278,  0.0168, -0.0264]],
       grad_fn=<ThAddmmBackward>)

In [43]:
model(**batch)

{'class_logits': tensor([[-0.0371,  0.0806, -0.0309,  0.0270,  0.0156, -0.0257],
         [-0.0385,  0.0808, -0.0315,  0.0275,  0.0149, -0.0257],
         [-0.0379,  0.0798, -0.0332,  0.0267,  0.0166, -0.0253],
         [-0.0387,  0.0787, -0.0305,  0.0272,  0.0173, -0.0241],
         [-0.0396,  0.0810, -0.0299,  0.0276,  0.0167, -0.0256],
         [-0.0390,  0.0815, -0.0305,  0.0276,  0.0160, -0.0249],
         [-0.0371,  0.0797, -0.0311,  0.0261,  0.0166, -0.0240],
         [-0.0378,  0.0807, -0.0309,  0.0268,  0.0151, -0.0257],
         [-0.0388,  0.0800, -0.0314,  0.0275,  0.0159, -0.0246],
         [-0.0379,  0.0802, -0.0313,  0.0283,  0.0164, -0.0240],
         [-0.0382,  0.0797, -0.0317,  0.0278,  0.0168, -0.0264]],
        grad_fn=<ThAddmmBackward>),
 'loss': tensor(0.6966, grad_fn=<MeanBackward1>)}

In [44]:
loss = model(**batch)["loss"]

In [45]:
loss

tensor(0.6966, grad_fn=<MeanBackward1>)

In [46]:
loss.backward()

# Train

In [47]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [48]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [49]:
metrics = trainer.train()


  0%|          | 0/5 [00:00<?, ?it/s]
loss: 0.6958 ||:  20%|██        | 1/5 [00:01<00:04,  1.04s/it]
loss: 0.6951 ||:  40%|████      | 2/5 [00:02<00:03,  1.25s/it]
loss: 0.6944 ||:  60%|██████    | 3/5 [00:03<00:02,  1.03s/it]
loss: 0.6934 ||:  80%|████████  | 4/5 [00:04<00:01,  1.08s/it]
loss: 0.6925 ||: 100%|██████████| 5/5 [00:05<00:00,  1.05it/s]
  0%|          | 0/5 [00:00<?, ?it/s]
loss: 0.6869 ||:  20%|██        | 1/5 [00:00<00:02,  1.43it/s]
loss: 0.6852 ||:  40%|████      | 2/5 [00:01<00:02,  1.19it/s]
loss: 0.6847 ||:  60%|██████    | 3/5 [00:02<00:01,  1.34it/s]
loss: 0.6831 ||:  80%|████████  | 4/5 [00:03<00:00,  1.33it/s]
loss: 0.6819 ||: 100%|██████████| 5/5 [00:04<00:00,  1.02s/it]

# Generating Predictions

AllenNLP is slightly lacking in its ability to convert datasets to predictions (though it has extensive support for converting single examples to predictions). Therefore, we'll write our own Predictor class to handle this job for us.

Thankfully, a lot of the tools we used eariler can easily be extended to prediction. Here's how.

In [50]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

We'll need a iterator that goes sequentially over our data. We'll use the 

In [51]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [52]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 


100%|██████████| 4/4 [00:00<00:00,  7.51it/s]

# A Final Note on Predictors

AllenNLP also provides predictors that take strings as input and outputs model predictions. They're handy if you want to create simple demo or need to make predictions on entirely new data, but since we've already read data as datasets and want to preserve their order, we didn't use them above. 

In [53]:
from allennlp.predictors.sentence_tagger import SentenceTaggerPredictor

In [54]:
tagger = SentenceTaggerPredictor(model, reader)

In [55]:
tagger.predict("this tutorial was great!")

{'class_logits': [-0.0763208195567131,
  0.03355295956134796,
  -0.06692716479301453,
  -0.0008314028382301331,
  -0.028824083507061005,
  -0.06674063950777054],
 'loss': 0.6763210892677307}

In [56]:
tagger.predict("this tutorial was horrible!")

{'class_logits': [-0.0763208195567131,
  0.03355295956134796,
  -0.06692716479301453,
  -0.0008314028382301331,
  -0.028824083507061005,
  -0.06674063950777054],
 'loss': 0.6763210892677307}